# 아파트 고유 식별자로 파일 생성

# load

In [1]:
unique_apt_group_file_name = '../data/unique_apt_group_v2.csv'

In [2]:
# sys.path를 수정하여 모듈 경로를 설정
import sys
import os

# 프로젝트 루트 경로 추가
project_root = os.path.abspath("..")
sys.path.append(project_root)

In [3]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# AppleGothic
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
import module

In [5]:
df = module.file_load.load_origin_data()

In [6]:
df['is_test'].value_counts()

0    1118822
1       9272
Name: is_test, dtype: int64

In [7]:
print(df.columns)
print(df.shape)

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test'],
      dtype='object')
(1128094, 53)


## 컬럼 정제

In [8]:
# 시, 군, 구 추가
module.parser.시_군_구_add(df)

In [12]:
# 서초포레스타2단지 내곡동 384
# 힐스테이트 서초 젠트리스	557
# 아파트 고유 식별자 추가
df.loc[df['아파트명'] == '서초포레스타2단지', '번지'] = df.loc[df['아파트명'] == '서초포레스타2단지', '번지'].fillna('384')
df.loc[df['아파트명'] == '힐스테이트 서초 젠트리스', '번지'] = df.loc[df['아파트명'] == '힐스테이트 서초 젠트리스', '번지'].fillna('557')
df['아파트명'] = df['아파트명'].fillna(df['도로명'])
module.parser.아파트_unique_key_add(df)

In [13]:
assert df['아파트_unique_key'].isnull().sum() == 0

In [14]:
df['아파트_unique_key'].value_counts().to_csv("아파트_unique_key.csv")

In [15]:
# 좌표X 기준으로 정렬 후, 첫 번째 값을 유지하여 채우는 함수
parse_df = df.copy()
module.parser.fill_coordinates_sorted(parse_df)

In [16]:
print(parse_df.shape)
print(parse_df.columns)

(9032, 57)
Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test', '시', '구', '동',
       '아파트_unique_key'],
      dtype='object')


## 저장

In [17]:
# 번지 빈값 2개 아파트명 대체
parse_df[parse_df['번지'].isnull()]
# 서초포레스타2단지 내곡동 384
# 힐스테이트 서초 젠트리스	557

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,시,구,동,아파트_unique_key


In [18]:
# (['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
#        '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
#        'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
#        'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
#        'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
#        'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
#        'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
#        '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
#        '좌표X', '좌표Y', '단지신청일', 'is_test', 'target'],
save_columns = ['아파트_unique_key','시', '구', '동','시군구', '번지', '본번','아파트명', '건축년도','도로명','좌표Y','좌표X']
parse_df[save_columns].to_csv(unique_apt_group_file_name, index=False)

In [ ]:
parse_df.head(1)